In [1]:
import esm
import torch
import pandas as pd
import tqdm

/data/nasif12/home_if12/l_schewinski/.conda/envs/l-schewinski-tp-4/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loadsm the model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
model.eval()

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bia

In [3]:
# get 5000 most important peptides
df = pd.read_csv('data/OsmoticStress_with_binary_positions_padded_5000.csv')

In [4]:
df_clean = df[["Uniprot_ID", "full_sequence", "Qvalue(LiP)"]]
df_clean.reset_index(drop=True, inplace=True)

In [5]:
df_clean.shape

(5000, 3)

In [6]:
# change to string the sequences and remove rows which doesent make sense
df_clean = df_clean[~df_clean['full_sequence'].apply(lambda x: isinstance(x, float))]
df_clean['full_sequence'] = df_clean['full_sequence'].astype(str)

In [7]:
df_clean

,Uniprot_ID,full_sequence,Qvalue(LiP)
0,P15703,MRFSTTLATAATALFFTASQVSAIGELAFNLGVKNNDGTCKSTSDY...,0.003686
1,P06169,MSEITLGKYLFERLKQVNVNTVFGLPGDFNLSLLDKIYEVEGMRWA...,0.003686
2,P38174,MTDAEIENSPASDLKELNLENEGVEQQDQAKADESDPVESKKKKNK...,0.003686
3,P00359,MVRVAINGFGRIGRLVMRIALSRPNVEVVALNDPFITNDYAAYMFK...,0.003686
5,Q04305,MSTARPRIITSKAPLLPQQTTPEQRYWRQYTSAQLVKEHNSVTHIS...,0.005126
...,...,...,...
4995,P27472,MSRDLQNHLLFETATEVANRVGGIYSVLKSKAPITVAQYKDHYHLI...,0.051166
4996,P20449,MSDTKRDPADLLASLKIDNEKEDTSEVSTKETVKSQPEKTADSIKP...,0.051166
4997,P30657,MNHDPFSWGRPADSTYGAYNTQIANAGASPMVNTQQPIVTGTSVIS...,0.051166
4998,P32501,MAGKKGQKKSGLGNHGKNSDMDVEDRLQAVVLTDSYETRFMPLTAV...,0.051166


In [8]:
# get the sequences of the unique protein id, so we wont generate twice the same embedding
df_clean_dropped = df_clean.drop_duplicates(subset='Uniprot_ID', keep='first')

In [9]:
df_clean_dropped

,Uniprot_ID,full_sequence,Qvalue(LiP)
0,P15703,MRFSTTLATAATALFFTASQVSAIGELAFNLGVKNNDGTCKSTSDY...,0.003686
1,P06169,MSEITLGKYLFERLKQVNVNTVFGLPGDFNLSLLDKIYEVEGMRWA...,0.003686
2,P38174,MTDAEIENSPASDLKELNLENEGVEQQDQAKADESDPVESKKKKNK...,0.003686
3,P00359,MVRVAINGFGRIGRLVMRIALSRPNVEVVALNDPFITNDYAAYMFK...,0.003686
5,Q04305,MSTARPRIITSKAPLLPQQTTPEQRYWRQYTSAQLVKEHNSVTHIS...,0.005126
...,...,...,...
4957,P36000,MPPLDKRIKKFLKDSIRIAPKISGKGELSELRTGLVSQYPQTRKDA...,0.051109
4959,P80428,MSNAALQVYGGDEVSAVVIDPGSYTTNIGYSGSDFPQSILPSVYGK...,0.051109
4964,P33441,MPLSQKQIDQVRTKVHYSEVDTPFNKYLDILGKVTKLTGSIINGTL...,0.051150
4987,P34240,MEKIPRWLLFSLISSVLCILGALCVPLLSVAFDSKRNSQSKLVNYG...,0.051166


In [23]:
import tqdm
import numpy as np
import torch

def generate_embeddings(model, alphabet, sequences):
    """
    Function to generate the embeddings.
    """
    embeddings = []
    
    batch_converter = alphabet.get_batch_converter()
    
    for sequence in tqdm.tqdm(sequences, desc="Generating Embeddings"):
        data = [(0, sequence)]
        batch_labels, batch_strs, batch_tokens = batch_converter(data)

        # Generate embeddings
        with torch.no_grad():
            results = model(batch_tokens, repr_layers=[33]) 
            token_embeddings = results['representations'][33]

        # Create full embeddings
        full_embedding = token_embeddings.squeeze().numpy()
        embeddings.append(full_embedding)
    
    return embeddings

In [24]:
def pad_sequence(sequence, target_length=1000, pad_token='<pad>'):
    """
    Add do the sequence <pad> to reach 1000.
    """
    padding_length = target_length - len(sequence)
    return sequence + pad_token * padding_length

# Apply padding to each sequence with <pad>
df_clean_dropped['padded_sequence'] = df_clean_dropped['full_sequence'].apply(lambda x: pad_sequence(x))

/data/nasif12/home_if12/l_schewinski/.conda/envs/l-schewinski-tp-4/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
# Check if the padding is correct
df_clean_dropped['full_sequence_length'] = df_clean_dropped['full_sequence'].apply(len)
df_clean_dropped['num_pad_tokens'] = df_clean_dropped['padded_sequence'].apply(lambda x: x.count('<pad>'))
df_clean_dropped['padded_sequence_length'] = df_clean_dropped['full_sequence_length'] + df_clean_dropped['num_pad_tokens']
df_clean_dropped['expected_padded_length'] = 1000

/data/nasif12/home_if12/l_schewinski/.conda/envs/l-schewinski-tp-4/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/data/nasif12/home_if12/l_schewinski/.conda/envs/l-schewinski-tp-4/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/data/nasif12/home_if12/l_schewinski/.conda/envs/l-schewinski-tp-4/lib/python3.7/site-packages/ipyk

In [24]:
df_clean_dropped.reset_index(drop=True, inplace=True)
df_clean_dropped = df_clean_dropped[1000:]

In [25]:
df_clean_dropped

,Uniprot_ID,full_sequence,padded_sequence,full_sequence_length,num_pad_tokens,padded_sequence_length,expected_padded_length
1000,Q04739,MAGDNPENKDASMLDVSDAASNTTINGKHSADSTNEASLAYTFSQM...,MAGDNPENKDASMLDVSDAASNTTINGKHSADSTNEASLAYTFSQM...,417,583,1000,1000
1001,P53172,MPQNTRHTSIVEMLSTPPQLPNSTDLNSLSEQTDKNTEANKSDTES...,MPQNTRHTSIVEMLSTPPQLPNSTDLNSLSEQTDKNTEANKSDTES...,527,473,1000,1000
1002,P33298,MEELGIVTPVEKAVEEKPAVKSYASLLAQLNGTVNNNSALSNVNSD...,MEELGIVTPVEKAVEEKPAVKSYASLLAQLNGTVNNNSALSNVNSD...,428,572,1000,1000
1003,Q12045,MASQQNKHAFLSKNRIFHNPDNVSSSKSRNLMDITNTTNTMNGSRP...,MASQQNKHAFLSKNRIFHNPDNVSSSKSRNLMDITNTTNTMNGSRP...,647,353,1000,1000
1004,Q03264,MTQDKEVKVVAPDVAPDQEVEINKSVKDAKHQTNDDSLLQHKKKGK...,MTQDKEVKVVAPDVAPDQEVEINKSVKDAKHQTNDDSLLQHKKKGK...,515,485,1000,1000
...,...,...,...,...,...,...,...
1471,Q03667,MARSRGSSRPISRSRPTQTRSASTMAAPVHPQQQQQPNAYSHPPAA...,MARSRGSSRPISRSRPTQTRSASTMAAPVHPQQQQQPNAYSHPPAA...,156,844,1000,1000
1472,Q12522,MATRTQFENSNEIGVFSKLTNTYCLVAVGGSENFYSAFEAELGDAI...,MATRTQFENSNEIGVFSKLTNTYCLVAVGGSENFYSAFEAELGDAI...,245,755,1000,1000
1473,P53875,MSQAAKNVIVKLIVGAGQAAPSPPVGPALGSKGIKAIDFCKEFNAR...,MSQAAKNVIVKLIVGAGQAAPSPPVGPALGSKGIKAIDFCKEFNAR...,158,842,1000,1000
1474,P38295,MSEVSKWPAINPFHWGYNGTVSHIVGENGSIKLHLKDNKEQVDFDE...,MSEVSKWPAINPFHWGYNGTVSHIVGENGSIKLHLKDNKEQVDFDE...,451,549,1000,1000


In [26]:
# Generate the embeddings!
sequences = df_clean_dropped['padded_sequence'].tolist()
embeddings = generate_embeddings(model, alphabet, sequences)

Generating Embeddings: 100%|██████████| 476/476 [1:13:50<00:00,  9.31s/it]


In [27]:
df_test_dropped['full_embedding'] = [e.tolist() for e in embeddings]

/data/nasif12/home_if12/l_schewinski/.conda/envs/l-schewinski-tp-4/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
# save embeddings
df_test_dropped.to_pickle('data/embeddings_new_3_3.pkl', protocol=4)